In [1]:
import jax
import jax.numpy as jnp
from jax import random, grad, vmap, jit
import neural_tangents as nt
from neural_tangents import stax
import optax

In [2]:
key = random.PRNGKey(42)
n_train, n_test = 20, 50
x_train = jnp.linspace(-jnp.pi, jnp.pi, n_train)[:, None]
y_train = jnp.sin(x_train) + 0.1 * random.normal(key, shape=(n_train, 1))
x_test = jnp.linspace(-jnp.pi, jnp.pi + 1, n_test)[:, None]
y_test = jnp.sin(x_test)

Platform 'METAL' is experimental and not all JAX functionality may be correctly supported!
W0000 00:00:1758505254.819535 1937444 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1758505254.912237 1937444 service.cc:145] XLA service 0x311608e50 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758505254.912359 1937444 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1758505254.920173 1937444 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1758505254.920190 1937444 mps_client.cc:384] XLA backend will use up to 5726240768 bytes on device 0 for SimpleAllocator.


Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [3]:
init_fn, apply_fn, kernel_fn = nt.stax.serial(
    stax.Dense(512, W_std=1.0, b_std=0.05),
    stax.Relu(),
    stax.Dense(1, W_std=1.0, b_std=0.05),
)
output_shape, params_init = init_fn(key, input_shape=x_train.shape)
predictions_init = apply_fn(params_init, x_train)